<a href="https://colab.research.google.com/github/louis-sanna-eki/structura/blob/master/notebooks/structura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install tqdm
!pip install nest_asyncio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
sample_submission_df = pd.read_json('drive/MyDrive/structura/sample_submission.json')
train_df = pd.read_parquet('drive/MyDrive/structura/train.parquet')
test_df = pd.read_parquet('drive/MyDrive/structura/test.parquet')

In [ ]:
json_data = {"0": "Accounting Policies and Estimates", "1": "Assets", "2": "Business Combinations", "3": "Cash Flow", "4": "Commitments and Contingencies", "5": "Employee Benefits", "6": "Equity", "7": "Expenses", "8": "Fair Value Measurements", "9": "Financial Instruments", "10": "Financial Performance", "11": "Financial Position", "12": "Intangible Assets", "13": "Investment and Equity Interests", "14": "Liabilities", "15": "Property, Plant, and Equipment", "16": "Regulatory Items", "17": "Related Party Transactions", "18": "Revenue", "19": "Risk Management", "20": "Segment Reporting", "21": "Shareholder Transactions", "22": "Taxation"}
nameToId = {v: k for k, v in json_data.items()}
classes = json_data.values()

# Convert to DataFrame
labels_df = pd.DataFrame(list(json_data.items()), columns=['Index', 'Label'])
json_data.values()

dict_values(['Accounting Policies and Estimates', 'Assets', 'Business Combinations', 'Cash Flow', 'Commitments and Contingencies', 'Employee Benefits', 'Equity', 'Expenses', 'Fair Value Measurements', 'Financial Instruments', 'Financial Performance', 'Financial Position', 'Intangible Assets', 'Investment and Equity Interests', 'Liabilities', 'Property, Plant, and Equipment', 'Regulatory Items', 'Related Party Transactions', 'Revenue', 'Risk Management', 'Segment Reporting', 'Shareholder Transactions', 'Taxation'])

In [ ]:
sample_submission_df.head()

,id,tokens,text,Accounting Policies and Estimates,Assets,Business Combinations,Cash Flow,Commitments and Contingencies,Employee Benefits,Equity,...,Investment and Equity Interests,Liabilities,"Property, Plant, and Equipment",Regulatory Items,Related Party Transactions,Revenue,Risk Management,Segment Reporting,Shareholder Transactions,Taxation
0,100,"[In, connection, with, disposal, groups, previ...",In connection with disposal groups previously ...,,,,,,,,...,,,,,,,,,,
1,101,"[The, variable, interest, rate, on, the, revol...",The variable interest rate on the revolving cr...,,,,,,,,...,,,,,,,,,,
2,102,"[As, of, September, 30, ,, 2019, ,, there, was...","As of September 30 , 2019 , there was no unrec...",,,,,,,,...,,,,,,,,,,
3,103,"[12, During, the, second, quarter, of, 2017, ,...","12 During the second quarter of 2017 , the Com...",,,,,,,,...,,,,,,,,,,
4,104,"[During, the, three, and, nine, months, ended,...",During the three and nine months ended Septemb...,,,,,,,,...,,,,,,,,,,


In [ ]:
labels_df.head()

,Index,Label
0,0,Accounting Policies and Estimates
1,1,Assets
2,2,Business Combinations
3,3,Cash Flow
4,4,Commitments and Contingencies


In [35]:
train_df.head()

,id,tokens,labels,text,Accounting Policies and Estimates,Assets,Business Combinations,Cash Flow,Commitments and Contingencies,Employee Benefits,...,Investment and Equity Interests,Liabilities,"Property, Plant, and Equipment",Regulatory Items,Related Party Transactions,Revenue,Risk Management,Segment Reporting,Shareholder Transactions,Taxation
0,5010,"[The, number, of, Common, Units, into, which, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",The number of Common Units into which vested A...,,,,,,,...,,,,,,,,,,
1,5011,"[At, September, 30, ,, 2019, ,, the, balance, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","At September 30 , 2019 , the balance of the ri...",,18.0,,,,,...,,,,,,,,,,
2,5012,"[As, a, result, of, adoption, ,, the, Company,...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1...","As a result of adoption , the Company recogniz...",,279.9,634.4,,,,...,,333.5,,,,,,,,
3,5013,"[Net, amortization, of, intangibles, ,, includ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","Net amortization of intangibles , including th...",,,,,,,...,,,,,,,,,,
4,5014,"[In, April, 2014, ,, the, San, Francisco, Vent...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","In April 2014 , the San Francisco Venture prov...",,,,,21.4 | 8.1,,...,,,,,,,,,,


In [36]:
selected_columns = train_df.columns[4:]

# Get unique values for each column
for col in selected_columns:
    unique_values = train_df[col].unique()
    print(f"Unique values of {col}: {unique_values}")

Unique values of Accounting Policies and Estimates: ['' '142' '8.4' '1.9' '9' '339' '5' '1.6' '163' '3.2' 'no' '40.3' '2.4'
 '0.3' '1.8' '1.0' '20' '5,800' '3.8 | 2.7' '104 | 108 | 4' '25'
 '8 | 10 | 1 | 2 | 1' '1.5' 'No' '3' '47' '0.2' '13.7' '86' '0.7' '2.3'
 '44,000' '5.2' '19.8' '23' '17.6' '13.3' '0.5' '340,000' '1.1' '145' '14'
 '78' '15.1 | 13.0 | 2.1' '57.4' '2.9' '65.3' '0.8' '8.3' '2,027' '0.1'
 '14.7' '12.1' '2.1' '12.8' '47.3' '24,682' '1.4' '136' '6' '331.0' '3.0'
 '0.6' '782' '2.7' '14.3' '22' '6.3' '21' '66.4' '84.6' '4.3 | 1.9' '126'
 '6,867' '4.9' '439' '146' '0.9' '2.6' '183' '7.9' '5,673' '8.2 | 8.2'
 '52.7' '21.0' '10.9' '17.1' '13 | 7' '1,057' '68 | 20' '64.1' '6,227'
 '4,176' '85,000' '1.3' '24' '18,000' '28.8' '4.1' '4' '306,000' '459,000'
 '11.2' '62.1' '7.1' '17.1 | 26.4 | 5.9 | 2.7 | 0.7' '3.3' '14.5'
 '71 | 168 | 56 | 183' '8,814' '7.8 | 265,000' '202.9' '8.0' '195.9'
 '2,145' '117.5' '2.2' '281' '513' '458' '23,013' '122,526' '1.7' '26.2'
 '486' '7.2' '84' '

In [32]:
train_df.loc[16, "text"]

'As of March 31 , 2020 , approximately C $ 33.0 million ( $ 23,261 ) of this facility was drawn and accruing interest at a rate of 2.80 % and approximately C $ 1.0 million ( $ 690 ) was supporting letters of credit leaving approximately C $ 26.0 million ( $ 18,341 ) available .'

In [ ]:
train_df.loc[2]

id                                                                                5012
tokens                               [As, a, result, of, adoption, ,, the, Company,...
labels                               [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1...
text                                 As a result of adoption , the Company recogniz...
Accounting Policies and Estimates                                                     
Assets                                                                           279.9
Business Combinations                                                            634.4
Cash Flow                                                                             
Commitments and Contingencies                                                         
Employee Benefits                                                                     
Equity                                                                                
Expenses                                   

### Parse

In [43]:
import re

def is_numeric(value):
    # Regular expressions to match numeric values
    number_pattern = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')

    # Dictionary of textual representations of numbers
    textual_numbers = {
        'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
        'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10,
        'no': 0  # Include 'no' if it represents zero
    }

    # Remove spaces and convert to lowercase for matching
    cleaned_value = value.replace(" ", "").lower()

    # Check if the value matches the number pattern or is in textual_numbers
    return bool(number_pattern.match(cleaned_value) or cleaned_value in textual_numbers)

# Example usage
test_strings = ['', '142', '8.4', 'no', '5,800', '23', 'four', 'zero', "minimun", "$$$"]
for string in test_strings:
    print(f"'{string}': {is_numeric(string)}")


'': False
'142': True
'8.4': True
'no': True
'5,800': True
'23': True
'four': True
'zero': True
'minimun': False
'$$$': False


In [37]:
from langchain.llms import OpenAI
import pandas as pd

llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key="...")

In [38]:
def parse(text):
    prompt = f"""
    ### Task
    The task is to extract financial KPIs then classify them.
    The classes are: ['Accounting Policies and Estimates', 'Assets', 'Business Combinations', 'Cash Flow', 'Commitments and Contingencies', 'Employee Benefits', 'Equity', 'Expenses', 'Fair Value Measurements', 'Financial Instruments', 'Financial Performance', 'Financial Position', 'Intangible Assets', 'Investment and Equity Interests', 'Liabilities', 'Property, Plant, and Equipment', 'Regulatory Items', 'Related Party Transactions', 'Revenue', 'Risk Management', 'Segment Reporting', 'Shareholder Transactions', 'Taxation']
    Your answer MUST be of the form (one line per KPI): `NUMBER=>CLASS\n`

    ### Example text
    As a result of adoption , the Company recognized approximately $ 279.9 million of operating ROU assets and approximately $ 333.5 million of operating lease liabilities as of January 1 , 2019 . NOTE 3 - ACQUISITIONS AND DISPOSITIONS Empire City Acquisition As discussed in Note 1 , on January 29 , 2019 , the Company acquired the developed real property associated with Empire City from MGM for fair value consideration of approximately $ 634.4 million .

    ### Example response
    279.9=>Assets
    333.5=>Liabilities
    634.4=>Business Combinations

    ### Example text
    The table below sets forth the components of assets held for sale on our consolidated balance sheet as of June 30 , 2019 for these properties ( in thousands ) : 21 2019 Dispositions On June 20 , 2019 , through a series of transactions , we sold a 90 % interest in seven data center shell properties in Virginia based on an aggregate property value of $ 265.0 million and retained a 10 % interest in the properties through BREIT COPT DC JV LLC ( ì BREIT - COPT î ) , a newly - formed joint venture .

    ### Example response
    10=>Investment and Equity Interests
    seven=>Assets
    265.0=>Financial Performance

    ### Example text
    The Company has three reportable segments as follows : Paperboard Mills includes the eight North American paperboard mills which produce primarily CRB , CUK , and SBS , which is consumed internally to produce paperboard packaging for the Americas and Europe Paperboard Packaging segments .

    ### Example response
    three=>Segment Reporting
    eight=>Assets

    ### Example text
    The excess of the principal amount of the liability component over its carrying amount ( the debt discount ) , along with related issuance fees are amortized to interest expense over the term of the Notes at an effective interest rate of 0.49 % . The net carrying amount of the liability component of the Notes is as follows : The net carrying amount of the equity component of the Notes is as follows : 69 Table of Contents The interest expense recognized related to the Notes is as follows : There was no liability or equity carrying amounts relating to the Notes as of June 24 , 2018 and no interest expense relating to the Notes was recognized for the fiscal years ended June 24 , 2018 and June 25 , 2017 . The estimated fair value of the convertible notes is $ 664.4 million , as determined by a Level 2 valuation as of June 30 , 2019 . Note 12 - Shareholders í Equity On June 14 , 2017 , the Board of Directors approved the Company 's fiscal 2018 stock repurchase program , authorizing the Company to repurchase shares of its common stock having an aggregate purchase price not exceeding $ 200.0 million for all purchases from June 26 , 2017 through the expiration of the program on June 24 , 2018 .

    ### Example response
    0.49=>Financial Instruments
    no=>Liabilities
    no=>Expenses
    664.4=>Fair Value Measurements
    200.0=>Shareholder Transactions

    ### Text
    {text}
    """

    completion_str = llm.invoke(prompt)
    completion_pairs = [line.split("=>") for line in completion_str.strip().split("\n")]
    kpis = []
    for pair in completion_pairs:
      if len(pair) != 2:
        continue
      value = pair[0]
      if value == "0.0":
        continue
      class_name = pair[1]
      if class_name in classes:
         kpis.append({ "value": value.strip(), "class_name": class_name  })
    return kpis

In [39]:
from tqdm import tqdm
import time

# Define a function to process rows in batches
def process_batch(df_batch):
    batch_results = []
    for index, row in df_batch.iterrows():
        text = row['text']
        id = row['id']
        kpis = parse(text)  # Synchronous call
        batch_results.append({"id": id, "text": text, "kpis": kpis})
    return batch_results

# Parameters
batch_size = 100  # Adjust as needed to avoid rate limits
sleep_time = 1  # Time to sleep between batches, adjust based on rate limits

# Initialize empty list for results
results = []

# Process in batches
for start in tqdm(range(0, len(test_df), batch_size)):
    end = start + batch_size
    df_batch = test_df.iloc[start:end]
    batch_results = process_batch(df_batch)
    results.extend(batch_results)
    time.sleep(sleep_time)  # Sleep to avoid hitting rate limits

# After this, results will contain the processed data


100%|██████████| 50/50 [35:35<00:00, 42.71s/it]


In [40]:
results

[{'id': 100,
  'text': 'In connection with disposal groups previously classified as held for sale , during 2018 , we recorded impairment charges totaling $ 28,817 to adjust the long - lived asset values within the Shanghai Golden Monkey ( " SGM " ) and Tyrrells disposal groups .',
  'kpis': []},
 {'id': 101,
  'text': 'The variable interest rate on the revolving credit facility was 9.5 % at March 31 , 2019 .',
  'kpis': [{'value': '9.5', 'class_name': 'Financial Instruments'}]},
 {'id': 102,
  'text': 'As of September 30 , 2019 , there was no unrecognized compensation cost related to stock - settled SARs outstanding . Cash - Settled SARs A summary of cash - settled SARs activity as of September 30 , 2019 and changes during the nine months then ended is as follows : As of September 30 , 2019 , all the cash - settled SARs are fully vested .',
  'kpis': [{'value': '0', 'class_name': 'Expenses'},
   {'value': 'no', 'class_name': 'Equity'},
   {'value': '0', 'class_name': 'Liabilities'},
  

In [45]:
import re

def remove_special_characters(s):
    # This regular expression pattern matches any character that is not a letter, digit, space, period, or comma
    pattern = re.compile('[^\w\s.,]')
    # Replace these characters with an empty string
    return pattern.sub('', s)

# Example usage
test_string = "Hello! This is a test string with % and $ and numbers like 1,23.45."
print(remove_special_characters(test_string))


Hello This is a test string with  and  and numbers like 1,23.45.


In [46]:
submission_df = sample_submission_df.copy(deep=True)

for result in results:
    id = result["id"]
    for kpi in result["kpis"]:
      class_name = kpi["class_name"]
      value = kpi["value"]
      value = remove_special_characters(value)
      existing_value = submission_df.loc[submission_df['id'] == id, class_name].iloc[0]

      if not is_numeric(value):
        continue

      if pd.isna(existing_value) or existing_value == "":
          # If there is no existing value, just set it
          submission_df.loc[submission_df['id'] == id, class_name] = value
      else:
          # If there is an existing value, append the new value with a separator
          submission_df.loc[submission_df['id'] == id, class_name] = f"{existing_value} | {value}"


In [47]:
submission_df.to_json("/content/final_output.json")


from google.colab import files

files.download('final_output.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
submission_df.shape

(4910, 26)

In [ ]:
sample_submission_df.shape

In [50]:
submission_df.loc[submission_df["id"] == 102]

,id,tokens,text,Accounting Policies and Estimates,Assets,Business Combinations,Cash Flow,Commitments and Contingencies,Employee Benefits,Equity,...,Investment and Equity Interests,Liabilities,"Property, Plant, and Equipment",Regulatory Items,Related Party Transactions,Revenue,Risk Management,Segment Reporting,Shareholder Transactions,Taxation
2,102,"[As, of, September, 30, ,, 2019, ,, there, was...","As of September 30 , 2019 , there was no unrec...",,,,,,,no,...,,0,,,,,,,,


In [ ]:
submission_df.loc[submission_df["id"] == 101].iloc[0, 2]

'The variable interest rate on the revolving credit facility was 9.5 % at March 31 , 2019 .'